In [ ]:
from pathlib import Path
import pandas as pd

# 설정
base_dir = Path("Results")
files = ["NV_1d_LinQuad.csv", "NV_5d_LinQuad.csv", "NV_20d_LinQuad.csv"]
loss_columns = ['loss_NF_GMM', 'loss_nonNF_GMM', 'loss_kannan', 'loss_nw']

rows = []

for fname in files:
    fpath = base_dir / fname

    df = pd.read_csv(fpath)

    cols_exist = [c for c in loss_columns if c in df.columns]

    q = df[cols_exist].quantile([0.1, 0.9])
    mean_row = pd.DataFrame(df[cols_exist].mean()).T
    mean_row.index = ['mean']
    stats = pd.concat([q, mean_row])

    stats = stats.reset_index().rename(columns={'index': 'stat'})
    stats['file'] = fname
    long = stats.melt(id_vars=['file', 'stat'], value_vars=cols_exist,
                      var_name='loss_col', value_name='value')
    rows.append(long)

if rows:
    result_long = pd.concat(rows, ignore_index=True)
    result_long['value'] = result_long['value'].round(3)

    result_wide = result_long.pivot_table(index=['file','stat'],
                                          columns='loss_col',
                                          values='value',
                                          aggfunc='first').reset_index()

    print("=== Combined quantiles (10%, 90%) and mean for each loss column ===")
    print(result_wide.to_string(index=False))

=== Combined quantiles (10%, 90%) and mean for each loss column ===
              file stat  loss_NF_GMM  loss_kannan  loss_nonNF_GMM  loss_nw
 NV_1d_LinQuad.csv  0.1        5.757       10.070           5.722    8.869
 NV_1d_LinQuad.csv  0.9       12.946       22.602          12.931   14.205
 NV_1d_LinQuad.csv mean        9.880       15.406           9.748   11.509
NV_20d_LinQuad.csv  0.1       94.249      113.379         103.331  196.473
NV_20d_LinQuad.csv  0.9      174.691      306.892         182.220  275.430
NV_20d_LinQuad.csv mean      140.706      196.875         143.365  233.464
 NV_5d_LinQuad.csv  0.1        7.549        9.459           8.694   15.756
 NV_5d_LinQuad.csv  0.9       45.351       50.481          61.826   56.568
 NV_5d_LinQuad.csv mean       29.518       31.060          34.986   37.003
